<a href="https://colab.research.google.com/github/i-ganza007/Multimodal-Data-Preprocessing/blob/main/Facial_Recognition_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import joblib
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, log_loss

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image

from PIL import Image
from scipy.spatial.distance import euclidean
import os


In [21]:
# STEP 1: Load Data
df = pd.read_csv("image_features.csv")
print(df.head())

# Extract labels and features
y = df['image'].apply(lambda name: name.split('_')[0])
X = df.drop(columns=['image'])

print(f"\n Total images in dataset: {len(X)}")


                    image         0         1         2         3         4  \
0    Placide_neutral.jpeg  0.000000  0.091155  0.088838  0.583058  0.000000   
1      Ian_surprised.jpeg  0.021418  1.188847  0.000000  0.309309  0.000000   
2  Placide_surprised.jpeg  0.000000  0.145360  0.069473  0.435874  0.143992   
3     Lievin_neutral.jpeg  0.260535  0.073728  0.000000  0.436296  0.244450   
4        Ian_smiling.jpeg  0.030604  0.619586  0.000000  0.162110  0.000000   

          5         6         7         8  ...      1270      1271      1272  \
0  0.002453  0.147236  0.254484  0.040733  ...  1.447928  1.303237  2.182407   
1  1.576542  0.072625  0.096814  0.407646  ...  2.966798  1.155986  1.286467   
2  0.000000  0.751911  0.413801  0.136104  ...  1.474377  0.937366  2.170682   
3  0.320405  0.394663  0.366991  0.008947  ...  1.101795  2.308778  3.318239   
4  0.818817  0.151410  0.567656  0.384127  ...  2.508587  1.101584  2.065963   

       1273      1274      1275      1276   

In [22]:
# STEP 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
# STEP 3: Train Logistic Regression
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [24]:
# STEP 4: Predictions & Metrics
y_pred = model.predict(X_test)
probs = model.predict_proba(X_test)

# Accuracy & F1
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
loss = log_loss(y_test, probs, labels=model.classes_)

# STEP 5: Print Everything
print("\n Classification Report:")
print(classification_report(y_test, y_pred))

print(f"\n Accuracy: {acc:.4f}")
print(f" F1 Score: {f1:.4f}")
print(f" Log Loss: {loss:.4f}")



 Classification Report:
              precision    recall  f1-score   support

        Eddy       1.00      1.00      1.00         3
         Ian       1.00      1.00      1.00         2
      Lievin       1.00      1.00      1.00         4
     Placide       1.00      1.00      1.00         1

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10


 Accuracy: 1.0000
 F1 Score: 1.0000
 Log Loss: 0.0122


In [14]:
# Save logistic regression model
joblib.dump(model, "face_recognition_model.pkl")

# Save training features & labels for distance check
with open("known_features.pkl", "wb") as f:
    pickle.dump((X_train.to_numpy(), y_train.to_numpy()), f)


In [27]:
# Load MobileNet model
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

def load_and_preprocess_image(img_path):
    try:
        img = Image.open(img_path).resize((224, 224))
        img_array = np.array(img)

        # Convert grayscale to RGB
        if img_array.ndim == 2:
            img_array = np.stack((img_array,) * 3, axis=-1)
        elif img_array.shape[2] == 1:
            img_array = np.repeat(img_array, 3, axis=-1)

        img_array = preprocess_input(img_array.astype(np.float32))
        img_array = np.expand_dims(img_array, axis=0)

        return img_array
    except Exception as e:
        print(" Error loading image:", e)
        return None


In [28]:
def test_on_unseen_face(img_path, distance_threshold=0.6):
    print(f"\n🧪 Testing on image: {os.path.basename(img_path)}")

    img_tensor = load_and_preprocess_image(img_path)
    if img_tensor is None:
        return

    # Load models and known data
    model = joblib.load("face_recognition_model.pkl")
    with open("known_features.pkl", "rb") as f:
        known_features, known_labels = pickle.load(f)

    # Step 1: Extract features
    feature_vector = mobilenet_model.predict(img_tensor)[0]

    # Step 2: Predict class using classifier
    probs = model.predict_proba([feature_vector])[0]
    predicted_index = np.argmax(probs)
    predicted_class = model.classes_[predicted_index]
    confidence = probs[predicted_index]

    print("\n Prediction Probabilities:")
    for cls, prob in zip(model.classes_, probs):
        print(f"{cls}: {prob:.2f}")

    print(f"\n Predicted: {predicted_class}")
    print(f" Confidence: {confidence:.2f}")

    # Step 3: Distance check
    distances = [euclidean(feature_vector, known_vec) for known_vec in known_features]
    min_distance = min(distances)
    closest_label = known_labels[np.argmin(distances)]

    print(f" Min Distance to known face: {min_distance:.4f}")
    print(f" Closest to: {closest_label}")

    if min_distance < distance_threshold:
        print(f" Access Granted to: {predicted_class}")
    else:
        print(" Access Denied: Unknown user")


In [30]:
test_on_unseen_face("ibinyomoro.jpeg")



🧪 Testing on image: ibinyomoro.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

 Prediction Probabilities:
Eddy: 0.05
Ian: 0.77
Lievin: 0.05
Placide: 0.13

 Predicted: Ian
 Confidence: 0.77
 Min Distance to known face: 25.4677
 Closest to: Ian
 Access Denied: Unknown user


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
